<a href="https://colab.research.google.com/github/GurkiratSarna/Information-Retrieval/blob/master/Assigment_1_1900690.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Indexing for Web Search
## Information Retrieval Assignment 1
### Registeration ID's :
### 1900690
### 

Installing and importing the required packages and libraries

In [0]:
# Umcomment and execute only once
#pip install spacy

In [0]:
# Umcomment and execute only once
#pip install inflect

In [0]:
import nltk, re
import sklearn
import urllib
import urllib.request
import urllib.parse
import numpy as np
import pandas as pd
import collections
import sys
from bs4 import BeautifulSoup
from nltk import sent_tokenize, word_tokenize
import inflect
import string, unicodedata
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk import pos_tag
from nltk import ne_chunk
from google.colab import files
import math

In [2]:
'''----Installing the nltk package----
please follow the prompted instructions after execution of this line of code for successfull download of nltk packages.
'''
nltk.download()
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [0]:
'''----Following are performed in the below code :
'''
def preprocessing(html_link):
  
  #----HTML Parsing----

  #Reading web page by calling read_web_page() function
  html=read_web_page(html_link)
  
  #Getting the plain text from the web page read above by calling get_html_text() function
  text=get_html_text(html)
  
  #----Pre-processing: Sentence Splitting, Tokenization and Normalization----

  #Split the sentence by calling sentence_splitting() function. It returns a list of strings separated by '. '.
  split=sentence_splitting(text)

  '''Tokenize the words in the sentences by using tokenize_word function. This function returns a list of lists after splitting the words in every
  sentence.
  '''
  list_token=[]
  list_token=tokenize_word(split)
  
  #Normalization
  norm_tokens = []
  for j in list_token:
    norm_token = normalize(j)
    norm_tokens.append(norm_token)
  
  #Convert list of lists to one lists
  norm_tokens_list = convert_listoflists(norm_tokens)
  #print(type(norm_tokens_list))
  #print(norm_tokens_list)

  #remove English Language stopwords - Fro example 'for', 'the', 'in' etc
  filtered_tokens = delete_stopwords(norm_tokens_list)
  #print(type(filtered_tokens))
  
  #----lemmatization and Stemming----
  
  # lemmas=[]
  #lemmatization
  lemmas = lemmatization(filtered_tokens)
  #print(type(lemmas))
  print("LEMMAS ]\n",lemmas)

  stems=[]
  #Stemming
  stems = stemming(lemmas)
  print("STEMS \n",stems)
  #print(type(stems))

  #----Part-of-Speech Tagging----
  tag = nltk.pos_tag(stems)
  print("TAGS \n",tag)
  #print(type(tag))
  
  #Entity recognition
  entity_reco_list = entity_reco(tag)
  #print("ENTITY RECO \n",entity_reco_list)
  return stems

In [0]:
def read_web_page(html_link):
  return urllib.request.urlopen(html_link).read().decode('utf8')

In [0]:
def get_html_text(html_link):
  
  soup = BeautifulSoup(html_link)
  
  '''---Delete script and style---
  This helps in removing the <script> and <style> html tag data and read that text as normal words. For example, when getting the text from the html_link
  'Media Memorability', the 'Contact' hyperlink at the end is not expanded to include the javascript as below-
  "var _rwObsfuscatedHref0 = "mai";var _rwObsfuscatedHref1 = "lto";var _rwObsfuscatedHref2 = ":m.";var _rwObsfuscatedHref3 = "a.l";
  var _rwObsfuscatedHref4 = "ars";var _rwObsfuscatedHref5 = "on@";var _rwObsfuscatedHref6 = "tud";var _rwObsfuscatedHref7 = "elf";
  var _rwObsfuscatedHref8 = "t.n";var _rwObsfuscatedHref9 = "l";  var _rwObsfuscatedHref = _rwObsfuscatedHref0+_rwObsfuscatedHref1+_rwObsfuscatedHref2+
  _rwObsfuscatedHref3+_rwObsfuscatedHref4+_rwObsfuscatedHref5+_rwObsfuscatedHref6+_rwObsfuscatedHref7+_rwObsfuscatedHref8+_rwObsfuscatedHref9; 
  document.getElementById('rw_email_contact').href = _rwObsfuscatedHref;"
  Instead it is read as normal text word 'Contact'.
  '''
  for script in soup(["script", "style"]):
    script.extract()
    
  #----Get all the text from the html_link passed to this function.----
  text = soup.get_text()
  
  #---- Remove leading and trailing spaces in each line----
  lines = (line.strip() for line in text.splitlines())
    
  #----break multi-headlines into each line----
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  
  #----remove blank lines and club the text----
  text = '\n'.join(chunk for chunk in chunks if chunk)
  
  return text

In [0]:
#----Sentence Splitting function. The tokenize function splits the text by '. ' and returns a list of these strings.----

def sentence_splitting(text):
  split = sent_tokenize(text)
  return split


In [0]:
def tokenize_word(split_list):
  list_token=[]
  for i in split_list:
    token = word_tokenize(i)
    list_token.append(token)
  return list_token

In [0]:
#Replacing numbers into characters. For example 9 becomes 'nine'.
def rep_numbers(token):
    p = inflect.engine()
    words = []
    for i in token:
        if i.isdigit():
            new_word = p.number_to_words(i)
            words.append(new_word)
        else:
            words.append(i)
    return words


#Change the all words to lowercase
def lowerc(token):
    words = []
    for i in token:
        nrm_word = i.lower()
        words.append(nrm_word)
    return words

#Remove punctuation
def punct(token):
    words = []
    for i in token:
        nrm_word = re.sub(r'[^\w\s]','', i)
        if nrm_word != '':
            words.append(nrm_word)
    return words

#Call the above three functions from one function, this function is called from the main function.
def normalize(token):
    token = rep_numbers(token)
    token = lowerc(token)
    token = punct(token)
    return token

In [0]:
def convert_listoflists(token_listoflists):
  norm_list = []
  for i in token_listoflists:
    for j in i:
        norm_list.append(j)
  return norm_list

In [0]:
def delete_stopwords(token_list):
  
  #List of English language Stopwords
  stop_words = set(stopwords.words('english'))

  #Filtering stopwords from the token list
  filtered = []
  for i in token_list:
    if i not in stop_words:
        filtered.append(i)
  return filtered

In [0]:
def lemmatization(filtered):
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    #print("in lemmatization")
    for i in filtered:
        #print("In for loop")
        lemma = lemmatizer.lemmatize(i, pos='v')
        lemmas.append(lemma)
    #print("\n in lemmatization, type of lemmas returned : ", type(lemmas))
    return lemmas

In [0]:
def stemming(lemmas):
    stemmer = LancasterStemmer()
    stems = []
    for i in lemmas:
        stem = stemmer.stem(i)
        stems.append(stem)
    return stems

In [0]:
#Entity recognition
def entity_reco(tag):
  chu = ne_chunk(tag, binary=True)
  return chu

In [0]:
def word_count(worddict, stemlist):
  for word in stemlist:
    worddict[word]+=1
  return worddict

In [0]:
#Creating the function to calculate the TF
def computeTF (wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word]=count/float(bowCount)
    return tfDict

In [0]:
#Create the fucntion to calculate the IDF
def computeIDF(docList):
    print("\n In computeIDF function")
    print(docList)
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict

In [0]:
#Create the function to calculate the TF.IDF score
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [0]:
def download_file(filename):
  filename=pd.DataFrame(filename)
  filename.to_csv('Select Name while downloading.csv')
  files.download('Select Name while downloading.csv')

In [19]:
'''----get a list of html links/ URL's from the user that need to be processed---
'''

html_link_list=[]
user_input='Yes'

while(user_input=='Yes' or user_input=='yes'):
  html_link_list.append(input("Please enter the web link : "))
  user_input=input("Do you wish to enter more html links ? (yes/NO) : ")

#Following can be uncommented to download the HTML lists. You will have a prompt to select the destination and the file name.
#download_file(html_link_list)

Please enter the web link : http://www.multimediaeval.org/mediaeval2019/memorability/
Do you wish to enter more html links ? (yes/NO) : yes
Please enter the web link : https://sites.google.com/view/siirh2020/
Do you wish to enter more html links ? (yes/NO) : 


In [26]:
# this is the list of all lists that will be returned after stemming. Each list belong to each webpage.
stem_list=[]
for i in range(len(html_link_list)):
  stem_list.append(preprocessing(html_link_list[i]))
print("\n STEMLIST", stem_list)

#create a set of words from the list of stems that is combine all the stem lists and sort it
set_of_words={}
for i in range(len(stem_list)):
  #for j in range(len(stem_list[i])):
  set_of_words=set(set_of_words).union(set(stem_list[i]))
set_of_words=sorted(set_of_words)
print(set_of_words)

#---------------------------
# Create a dictionary of dictionary type variables. The number of variables depends on the number of html links provided initially or the length of the stem list any which way.
d={}
for x in range(len(stem_list)):
        d["wordDict{0}".format(x)]={}
print("created dictionary d")
print(d)
#----------------------------
#Each dictionary variable created above is assigned the set of words with 0 value so it is easy to calculate the occurrences to calculate tf-idf
for key in d.keys():
  d[key]=dict.fromkeys(set_of_words,0)

print(d)
print("--"*20)

LEMMAS ]
 ['media', 'memorability', 'mediaeval', 'benchmarking', 'initiative', 'multimedia', 'evaluation', 'multi', 'multimedia', 'speech', 'audio', 'visual', 'content', 'tag', 'users', 'context', 'homeabout', 'mediaevaldatasetsmediaeval', '2019emotion', 'theme', 'musiceyes', 'ears', 'togethergamestorylifelogging', 'wellbeingmedico', 'multimediamultimedia', 'recsysmultimedia', 'satellitenoaudio', 'speech', 'turnspixel', 'privacymedia', 'memorabilityscene', 'changesports', 'videomediaeval', '2018mediaeval', '2017mediaeval', '2016mediaeval', '2015mediaeval', '2014mediaeval', '2013mediaeval', '2012mediaeval', '2011mediaeval', '2010videoclef', '2009videoclef', '2008videos', 'mediaevalwhy', 'participate', 'open', 'scienceacknowledgments', 'two thousand and nineteen', 'predict', 'media', 'memorability', 'tasktask', 'descriptionthis', 'task', 'focus', 'problem', 'predict', 'memorable', 'video', 'viewers', 'require', 'participants', 'automatically', 'predict', 'memorability', 'score', 'videos'

In [28]:
#For every dictionary belonging to every html link, call the word count function which provides the count for each word in its corresponding stem
stem_list_index=0
for key, dval in d.items():
  #print(dval)
  #print(stem_list[stem_list_index])
  d[key]=word_count(dval,stem_list[stem_list_index])
  stem_list_index +=1


print("\n After Word Count dictionary d :")
print(d)

#For displaying properly as a data frame
test=pd.DataFrame(d)
test=test.transpose()
display(test)


 After Word Count dictionary d :
{'wordDict0': {'10000': 1, '1159pm': 0, '14691482': 1, '16921703': 1, '2008videos': 1, '2009videoclef': 1, '2010videoclef': 1, '2011mediaeval': 1, '2012mediaeval': 1, '2013mediaeval': 1, '2014mediaeval': 1, '2015mediaeval': 1, '2016mediaeval': 1, '2017mediaeval': 1, '2018mediaeval': 1, '2019emotion': 1, '2019runs': 1, '2019test': 1, '2019working': 1, '2020camera': 0, '2020notificationsfull': 0, '2020short': 0, '23902398': 1, '2729': 1, '27302739': 1, '2931': 1, '42nd': 0, 'abl': 2, 'abstract': 0, 'abus': 0, 'abusep': 0, 'acceiv': 0, 'accompany': 1, 'acm': 1, 'addit': 0, 'address': 1, 'adity': 1, 'adv': 0, 'advert': 1, 'aesthet': 2, 'affect': 1, 'akhil': 2, 'alamedapined': 1, 'alberto': 0, 'alexnet': 1, 'algorithm': 1, 'allow': 1, 'already': 1, 'also': 2, 'among': 0, 'amount': 1, 'analys': 4, 'analysis': 1, 'annot': 2, 'annount': 0, 'answ': 0, 'antipol': 1, 'antonio': 2, 'apply': 2, 'approach': 0, 'april': 0, 'are': 0, 'area': 3, 'arxiv181201973please':

,10000,1159pm,14691482,16921703,2008videos,2009videoclef,2010videoclef,2011mediaeval,2012mediaeval,2013mediaeval,2014mediaeval,2015mediaeval,2016mediaeval,2017mediaeval,2018mediaeval,2019emotion,2019runs,2019test,2019working,2020camera,2020notificationsfull,2020short,23902398,2729,27302739,2931,42nd,abl,abstract,abus,abusep,acceiv,accompany,acm,addit,address,adity,adv,advert,aesthet,...,usathierry,usayonghu,usayoshinobu,usazit,valenc,vast,venu,via,video,videomediaev,videomem,view,vigo,villega,villetaneus,vis,vocab,vorh,way,webcont,wei,wel,wellbeingmedico,western,work,workshop,workshopa,workshopsearch,workshopsem,writ,wu,wwwspringercomgpcomputersciencelncsconferenceproceedingsguidelineseasychair,wwwtechnicolorcomdreamresearchinnovationmoviememorabilitydatasettask,xavy,xiao,xintao,yadat,year,zurich,ştef
wordDict0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,0,2,0,0,0,0,1,1,0,1,1,0,1,2,...,0,0,0,0,0,0,0,1,22,1,1,2,0,0,0,5,0,0,1,0,0,2,1,0,2,2,0,0,0,0,0,0,1,1,1,1,1,2,0,1
wordDict1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,4,1,1,1,0,0,1,0,0,1,0,0,...,1,1,1,1,1,1,2,0,0,0,0,0,1,1,1,0,1,1,0,1,1,1,0,1,1,7,1,1,1,1,1,1,0,0,0,0,0,0,1,0


In [29]:

stem_list_index=0
TFBow={**d}
print(id(d))
print(id(TFBow))
print("\n After copying d into TFBow : ")
print(TFBow)

#Call computeTF function to for each dictionary value.
for key, TFBowval in TFBow.items():
  #print(TFBowval)
  #print(stem_list[stem_list_index])
  TFBow[key]=computeTF(TFBowval,stem_list[stem_list_index])
  stem_list_index +=1
print("---------------------")
print(TFBow)
print("---------------------")

#For displaying properly as a data frame
test=pd.DataFrame(TFBow)
test=test.transpose()
display(test)

140551036957536
140551035491456

 After copying d into TFBow : 
{'wordDict0': {'10000': 1, '1159pm': 0, '14691482': 1, '16921703': 1, '2008videos': 1, '2009videoclef': 1, '2010videoclef': 1, '2011mediaeval': 1, '2012mediaeval': 1, '2013mediaeval': 1, '2014mediaeval': 1, '2015mediaeval': 1, '2016mediaeval': 1, '2017mediaeval': 1, '2018mediaeval': 1, '2019emotion': 1, '2019runs': 1, '2019test': 1, '2019working': 1, '2020camera': 0, '2020notificationsfull': 0, '2020short': 0, '23902398': 1, '2729': 1, '27302739': 1, '2931': 1, '42nd': 0, 'abl': 2, 'abstract': 0, 'abus': 0, 'abusep': 0, 'acceiv': 0, 'accompany': 1, 'acm': 1, 'addit': 0, 'address': 1, 'adity': 1, 'adv': 0, 'advert': 1, 'aesthet': 2, 'affect': 1, 'akhil': 2, 'alamedapined': 1, 'alberto': 0, 'alexnet': 1, 'algorithm': 1, 'allow': 1, 'already': 1, 'also': 2, 'among': 0, 'amount': 1, 'analys': 4, 'analysis': 1, 'annot': 2, 'annount': 0, 'answ': 0, 'antipol': 1, 'antonio': 2, 'apply': 2, 'approach': 0, 'april': 0, 'are': 0, 'are

,10000,1159pm,14691482,16921703,2008videos,2009videoclef,2010videoclef,2011mediaeval,2012mediaeval,2013mediaeval,2014mediaeval,2015mediaeval,2016mediaeval,2017mediaeval,2018mediaeval,2019emotion,2019runs,2019test,2019working,2020camera,2020notificationsfull,2020short,23902398,2729,27302739,2931,42nd,abl,abstract,abus,abusep,acceiv,accompany,acm,addit,address,adity,adv,advert,aesthet,...,usathierry,usayonghu,usayoshinobu,usazit,valenc,vast,venu,via,video,videomediaev,videomem,view,vigo,villega,villetaneus,vis,vocab,vorh,way,webcont,wei,wel,wellbeingmedico,western,work,workshop,workshopa,workshopsearch,workshopsem,writ,wu,wwwspringercomgpcomputersciencelncsconferenceproceedingsguidelineseasychair,wwwtechnicolorcomdreamresearchinnovationmoviememorabilitydatasettask,xavy,xiao,xintao,yadat,year,zurich,ştef
wordDict0,0.001429,0.000000,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.000000,0.000000,0.000000,0.001429,0.001429,0.001429,0.001429,0.000000,0.002857,0.00000,0.000000,0.000000,0.000000,0.001429,0.001429,0.000000,0.001429,0.001429,0.000000,0.001429,0.002857,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001429,0.031429,0.001429,0.001429,0.002857,0.000000,0.000000,0.000000,0.007143,0.000000,0.000000,0.001429,0.000000,0.000000,0.002857,0.001429,0.000000,0.002857,0.002857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001429,0.001429,0.001429,0.001429,0.001429,0.002857,0.000000,0.001429
wordDict1,0.000000,0.004942,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001647,0.001647,0.001647,0.000000,0.000000,0.000000,0.000000,0.001647,0.000000,0.00659,0.001647,0.001647,0.001647,0.000000,0.000000,0.001647,0.000000,0.000000,0.001647,0.000000,0.000000,...,0.001647,0.001647,0.001647,0.001647,0.001647,0.001647,0.003295,0.000000,0.000000,0.000000,0.000000,0.000000,0.001647,0.001647,0.001647,0.000000,0.001647,0.001647,0.000000,0.001647,0.001647,0.001647,0.000000,0.001647,0.001647,0.011532,0.001647,0.001647,0.001647,0.001647,0.001647,0.001647,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001647,0.000000


In [37]:
#Computing the IDF for both documents
print(d)
print(TFBow)

#Create a list of dictionary variables
temp_list=[]
for k, v in d.items():
  temp_list.append(d[k])
print(temp_list)
idfs = computeIDF(temp_list)
print("\n IDFS : ")
print(idfs)

{'wordDict0': {'10000': 1, '1159pm': 0, '14691482': 1, '16921703': 1, '2008videos': 1, '2009videoclef': 1, '2010videoclef': 1, '2011mediaeval': 1, '2012mediaeval': 1, '2013mediaeval': 1, '2014mediaeval': 1, '2015mediaeval': 1, '2016mediaeval': 1, '2017mediaeval': 1, '2018mediaeval': 1, '2019emotion': 1, '2019runs': 1, '2019test': 1, '2019working': 1, '2020camera': 0, '2020notificationsfull': 0, '2020short': 0, '23902398': 1, '2729': 1, '27302739': 1, '2931': 1, '42nd': 0, 'abl': 2, 'abstract': 0, 'abus': 0, 'abusep': 0, 'acceiv': 0, 'accompany': 1, 'acm': 1, 'addit': 0, 'address': 1, 'adity': 1, 'adv': 0, 'advert': 1, 'aesthet': 2, 'affect': 1, 'akhil': 2, 'alamedapined': 1, 'alberto': 0, 'alexnet': 1, 'algorithm': 1, 'allow': 1, 'already': 1, 'also': 2, 'among': 0, 'amount': 1, 'analys': 4, 'analysis': 1, 'annot': 2, 'annount': 0, 'answ': 0, 'antipol': 1, 'antonio': 2, 'apply': 2, 'approach': 0, 'april': 0, 'are': 0, 'area': 3, 'arxiv181201973please': 1, 'aspect': 1, 'assess': 0, 'ass

In [0]:

#Calculating the TF.IDF score for both documents
tfidfBowA = computeTFIDF(tfBowA, idfs)
tfidfBowB = computeTFIDF(tfBowB, idfs)

#Data frame of the scores
score = pd.DataFrame([tfidfBowA, tfidfBowB])
print(score)

#Sort the scores (to know what words are more relevant in each document)

#Document 1 (Web page 1)
score1 = pd.DataFrame([tfidfBowA])
scorest1 = score1.T
scorest1.columns = ['tf.idf(doc1)']

scorestf1 = scorest1.sort_values(['tf.idf(doc1)'], ascending = False)
print(scorestf1)

#-------

#Document 2 (Web page 2)
score2 = pd.DataFrame([tfidfBowB])
scorest2 = score2.T
scorest2.columns = ['tf.idf(doc2)']

scorestf2 = scorest2.sort_values(['tf.idf(doc2)'], ascending = False)
print(scorestf2)

In [0]:
#Exportin the results to csv

scorestf1.to_csv('/Users/davidalbuja/Documents/Essex/Spring/Information Retrieval/Assigment 1/Outputs/TFIDF(doc1).csv')
scorestf2.to_csv('/Users/davidalbuja/Documents/Essex/Spring/Information Retrieval/Assigment 1/Outputs/TFIDF(doc2).csv') 